## Código de preparación de datos...

Este pequeño código prepara los datos para ser manipulados y ser usados por otro código de propagación de muones en roca.. La idea es que tenemos dos archivos de datos. El primero es la salida de muones del volcán machin en el punto P1 de observación, el otro archivo de datos es la distancia recorrida por los muones en roca, calculado a partir de un código en matlab externo. La idea es manipular el primero de los archivos para obtener la energía total a partir de la consideración de los momentos en las direcciones $x$, $y$, y $z$. Lo segundo es teniendo la energía vamos a organizar los datos de tal manera de calcular el flujo, es decir, sumar todos los muones que llegan en la misma dirección para tener el número de muones que entran por esa determinada dirección, con esto tendremos una data de flujo por dirección.
Lo siguiente es combinar el archivo de datos de distancias recorridas y le ponga a cada partícula entrante con energía determinada E, su correspondiente distancia recorrida...

In [1]:
# Cargando todas las librerías que necesitamos
import numpy as np
import math
import pandas as pd 
import matplotlib.pyplot as plt 
df1=pd.read_csv('./salida_d_2.shw',sep =" ", header=None)
df1=df1.drop(index=range(6))  
df2 = df1[df1.columns[1:4]] 
px = df2[df2.columns[:1]].to_numpy()
py = df2[df2.columns[1:2]].to_numpy()
pz = df2[df2.columns[2:3]].to_numpy()
pt=np.sqrt(px**2+py**2+pz**2) 

Llamamos al primer archivo de datos originales que vienen de Corsika...

Acá ahora tomamos los momentos y calculamos la energía total de cada muon que llega al volcán... En este caso 
\begin{equation}
\theta=\arccos\frac{p_z}{p}=\arccos\frac{p_z}{\sqrt{p_x^2+p_y^2+p_z^2}}, \ \ \phi=\arctan\frac{p_y}{p_x}
\end{equation}

In [2]:
pt=np.sqrt(px**2+py**2+pz**2)
m_rest=0.1057 #unit: GeV/c**2
Et=np.sqrt(pt**2+m_rest**2)
thetat=np.degrees(np.arccos(pz/pt))
at2=np.arctan2(py,px)
thetat=np.degrees(np.arccos(pz/pt))
at2=np.arctan2(py,px)
phit=np.degrees((2*np.pi+at2)*(at2<0)+(at2)*(at2>0))

Ahora tenemos un archivo de ángulos de incidencia vs. energía con la que llegan. Guardamos los datos nuevos con el formato requerido... Cargamos la nueva dataframe y ordenamos por ángulos.

In [3]:
data=np.c_[phit,thetat,Et] #theta ya esta en grados
data=data[data[:,1].argsort()] # Ordena de acuerdo a angulo theta 
print(data[-5:])

[[3.03538196e+02 8.99277464e+01 1.05734042e-01]
 [9.43142232e+00 8.99595315e+01 1.05701232e-01]
 [1.95272027e+02 8.99624061e+01 1.05701102e-01]
 [9.50055892e+01 8.99897996e+01 1.05724522e-01]
 [3.49913440e+02 8.99944578e+01 1.05705134e-01]]


In [5]:
theta1, theta2= 9, 26
#arrf = thetat[(thetat >= theta1) & (thetat<=theta2)] #thetat es el valor de ángulo azimutal
#print(type(arrf[0]))
#plt.hist(arrf, bins=50)

In [7]:
# Obtenemos solo los que están en el rango 117<phi<147 y 66<theta<84
data=data[(data[:,0]>=89) & (data[:,0]<=91) & (data[:,1]>=theta1) & (data[:,1]<=theta2)] #--- acá le pongo la condición 
data=np.savetxt('totalrealCerroUNI_tpE_ordenados.out',data,delimiter=' ',fmt='%i %i %2f')


In [10]:
data=np.savetxt('totalrealCerroUNI_tpE_ordenados.out',data,delimiter=' ',fmt='%i %i %2f')
df2=pd.read_csv('totalrealCerroUNI_tpE_ordenados.out',sep=' ',names=['phi','theta','E'])

ValueError: Expected 1D or 2D array, got 0D array instead

In [11]:
!head(totalrealMachin_tpE_ordenados.out)

/bin/bash: -c: línea 0: error sintáctico cerca del elemento inesperado `totalrealMachin_tpE_ordenados.out'
/bin/bash: -c: línea 0: `head(totalrealMachin_tpE_ordenados.out)'


Ahora agrupamos todos los muones que tengan la misma dirección de incidencia y con esto tenemos el flujo por ángulo.

In [12]:
df3=df2.groupby(['phi','theta'])['E'].count().reset_index(name='count')
df3.to_csv('totalrealMachin_tpcount_ordenados.out',sep=' ',index=False)
df3

KeyError: 'phi'

In [ ]:
p4 = df3[df3.columns[2:3]].to_numpy()
plt.hist(p4)

Cargamos el archivo de las distancias..

In [ ]:
df4=pd.read_csv('distp1_machin.out',sep='\t',names=['phi','theta','d'],skiprows=3)
df4

Y ahora combinamos las energías para cada muon en cada dirección de incidencia con las distancias recorridas en esas mismas incidencias...

In [ ]:
m = df4.merge(df2, on=['phi', 'theta'], how='outer')
m.to_csv('totalrealMachin_tpdE_ordenados.out',sep=' ',index=False)
m

Preparando el archivo final de entrada para el código de propagación...

In [ ]:
muones_to_propagate=np.loadtxt('totalrealMachin_tpdE_ordenados.out',delimiter=' ',skiprows=2)
#print muones_to_propagate

In [ ]:
dis=muones_to_propagate[:,2]*100 # Convert to cm, originally in m, code and tables use cm
ener=muones_to_propagate[:,3]*1000 # Convert to MeV, originally in GeV, code and tables use MeV
muones_to_propagate=np.c_[dis,ener]
#print muones_to_propagate
np.savetxt('muones_to_propagate.out',muones_to_propagate,delimiter=' ',fmt='%i %2f')

Para una pequeña prueba vamos a quedarnos solo con los indices múltiplos de 30...

In [ ]:
muones_to_propagate=muones_to_propagate[(muones_to_propagate[:,0]>0)]
muones_to_propagate_prueba=muones_to_propagate[1::100]
#muones_to_propagate_prueba=muones_to_propagate_prueba[::2]
#muones_to_propagate_prueba=muones_to_propagate_prueba[::2]
np.savetxt('muones_to_propagate_prueba100.out',muones_to_propagate_prueba,delimiter=' ',fmt='%i %2f')

In [ ]:
print np.shape(muones_to_propagate_prueba)